In [248]:
from __future__ import annotations

import argparse
import json
import os
import re
import sys
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional

import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [249]:
try:
    from sentence_transformers import SentenceTransformer
except Exception as e:  # pragma: no cover
    raise RuntimeError(
        "sentence-transformers is required. Install with: pip install sentence-transformers"
    ) from e

In [250]:
# ===== Notebook config =====
INPUT_FILE = "output/supertest_confessions_of_hnmu.xlsx"
OUTPUT_FILE = "output/output_supertest_confessions_of_hnmu.json"
MODEL = "VoVanPhuc/sup-SimCSE-VietNamese-phobert-base"

CACHE_DIR = None   # hoặc None
LOCAL_ONLY = True

MIN_SIM = 0.38
MARGIN = 0.05
HIGH_CONF = 0.55
OOS_GUARD_DELTA = 0.02
BATCH_SIZE = 32


In [251]:
TAXONOMY_8: Dict[str, Dict[str, List[str]]] = {
    "Đăng ký học phần": {
        "keywords": [
            "đăng ký học phần", "đăng ký môn", "đăng ký tín chỉ", "đăng kí tín chỉ",
            "chọn lớp", "chọn nhóm", "hủy đăng ký", "huỷ đăng ký", "rút học phần", "rút môn",
            "đăng ký lại", "học phần tiên quyết", "tiên quyết", "song hành", "học phần điều kiện",
            "mở lớp", "hủy lớp", "đóng lớp", "đổi lớp"
        ],
        "anchors": [
            "cách đăng ký học phần, đăng ký tín chỉ, chọn lớp và điều kiện tiên quyết",
            "rút học phần, hủy đăng ký môn, đăng ký lại học phần khi bị trùng lịch",
            "không đăng ký được học phần vì tiên quyết/song hành, xử lý như thế nào"
        ],
    },
    "Đánh giá - Chấm điểm": {
        "keywords": [
            "đánh giá", "chấm điểm", "thang điểm", "điểm quá trình", "điểm chuyên cần",
            "điểm giữa kỳ", "điểm cuối kỳ", "điểm tổng kết", "điểm thành phần", "gpa", "điểm chữ",
            "phúc khảo", "chấm lại", "nhập điểm", "sai điểm", "đề nghị sửa điểm"
        ],
        "anchors": [
            "quy định đánh giá và chấm điểm: điểm chuyên cần, điểm quá trình, điểm thi, điểm tổng kết",
            "phúc khảo bài thi, chấm lại, khi phát hiện sai điểm cần làm thủ tục gì",
            "cách tính GPA và điểm chữ, quy đổi điểm và điều kiện đạt môn"
        ],
    },
    "Học lại - Học cải thiện": {
        "keywords": [
            "học lại", "học cải thiện", "cải thiện điểm", "học lại môn", "đăng ký học lại",
            "đăng ký cải thiện", "rớt môn", "trượt môn", "học lại để qua môn", "xóa điểm", "xoá điểm"
        ],
        "anchors": [
            "quy định học lại và học cải thiện: khi nào được đăng ký và cách tính điểm",
            "trượt môn/rớt môn thì đăng ký học lại ra sao, điểm cũ và điểm mới được tính thế nào",
            "cải thiện điểm để tăng GPA, quy tắc thay thế điểm"
        ],
    },
    "Cảnh báo học vụ - Thôi học": {
        "keywords": [
            "cảnh báo học vụ", "cảnh báo", "thôi học", "buộc thôi học",
            "đình chỉ học", "đình chỉ", "học tiếp", "mất quyền học tiếp",
            "nợ tín chỉ", "tín chỉ tích lũy", "không đủ điều kiện", "gpa thấp", "trượt nhiều"
        ],
        "anchors": [
            "cảnh báo học vụ theo GPA/tín chỉ tích lũy, điều kiện để tiếp tục học",
            "các mức xử lý học vụ: cảnh báo, đình chỉ, buộc thôi học và cách khắc phục",
            "mất quyền học tiếp do kết quả học tập, quy trình xét và khiếu nại"
        ],
    },
    "Học phí - Miễn giảm": {
        "keywords": [
            "học phí", "đóng học phí", "nợ học phí", "miễn giảm", "miễn học phí", "giảm học phí",
            "hoàn học phí", "hoàn tiền", "biên lai", "hóa đơn", "thu học phí", "mức học phí",
            "phí tín chỉ", "phí học phần"
        ],
        "anchors": [
            "quy định học phí: mức thu, cách tính theo tín chỉ/học phần và thời hạn đóng",
            "miễn giảm học phí theo đối tượng và hồ sơ cần nộp",
            "nợ học phí ảnh hưởng đăng ký học phần/kết quả học tập như thế nào"
        ],
    },
    "Học bổng - Khen thưởng": {
        "keywords": [
            "học bổng", "xét học bổng", "học bổng khuyến khích", "tiêu chí học bổng",
            "khen thưởng", "giải thưởng", "hen thưởng", "trợ cấp", "hỗ trợ", "tài trợ",
            "điểm rèn luyện", "điểm rl"
        ],
        "anchors": [
            "điều kiện và tiêu chí xét học bổng khuyến khích học tập, hồ sơ và thời gian xét",
            "khen thưởng sinh viên có thành tích, tiêu chuẩn và quy trình xét duyệt",
            "học bổng tài trợ/trợ cấp, điều kiện nhận và cách nộp hồ sơ"
        ],
    },
    "Thực tập - Khóa luận": {
        "keywords": [
            "thực tập", "intern", "thực tập tốt nghiệp", "báo cáo thực tập", "nhật ký thực tập",
            "khóa luận", "khoá luận", "đồ án", "đồ án tốt nghiệp", "đề cương", "đề tài",
            "giảng viên hướng dẫn", "hội đồng", "bảo vệ", "bảo vệ khóa luận"
        ],
        "anchors": [
            "quy định thực tập và khóa luận/đồ án: điều kiện, thời gian, hồ sơ và cách đánh giá",
            "chọn đề tài, đăng ký giảng viên hướng dẫn, nộp đề cương và báo cáo thực tập",
            "bảo vệ khóa luận/đồ án, hội đồng chấm và tiêu chí đánh giá"
        ],
    },
    "Xét và công nhận tốt nghiệp": {
        "keywords": [
            "xét tốt nghiệp", "xét & công nhận tốt nghiệp", "công nhận tốt nghiệp", "tốt nghiệp",
            "điều kiện tốt nghiệp", "chuẩn đầu ra", "chứng chỉ", "ngoại ngữ", "tin học",
            "nợ môn", "hoàn thành", "bằng tốt nghiệp", "cấp bằng", "ra trường"
        ],
        "anchors": [
            "điều kiện xét và công nhận tốt nghiệp: tín chỉ, GPA, chuẩn đầu ra và các chứng chỉ",
            "quy trình xét tốt nghiệp, thời hạn nộp hồ sơ và xử lý trường hợp còn nợ môn",
            "cấp bằng tốt nghiệp, bảng điểm và các thủ tục sau khi được công nhận"
        ],
    },
}

TOPIC_ORDER: List[str] = list(TAXONOMY_8.keys())

In [252]:
# Strong OOS cues for social/personal, buy-sell, rental, love, lost & found, chit-chat
OOS_KEYWORDS = [
    # greetings / chit-chat
    "chào", "hi", "hello", "mn ơi", "mọi người ơi", "cho mình hỏi ngoài lề", "tâm sự", "confession",
    # rental / roommate
    "tìm trọ", "roommate", "ở ghép", "tìm phòng", "chung cư", "nhà trọ", "pass phòng",
    # buy/sell
    "mua bán", "thanh lý", "pass đồ", "bán", "mua", "order", "ship", "giveaway",
    # love/relationship
    "crush", "ny", "người yêu", "tình cảm", "chia tay", "tỏ tình",
    # lost & found (your example)
    "nhặt được", "nhặt đc", "nhặt dc", "đánh rơi", "rơi", "mất ví", "mất thẻ", "thẻ sinh viên",
    "liên hệ mình", "ai đánh rơi", "ai làm rơi", "trả lại", "gửi lại",
]

OOS_ANCHORS = [
    "câu hỏi xã giao", "chào hỏi", "trò chuyện cá nhân", "tâm sự không liên quan học vụ",
    "tìm trọ", "tìm roommate", "mua bán thanh lý đồ", "pass đồ", "chuyện tình cảm",
    "nhặt được thẻ sinh viên hoặc đồ vật", "nhờ liên hệ để trả lại", "đánh rơi thẻ sinh viên hoặc đồ vật",
    "nhặt được thẻ sinh viên", "đánh rơi thẻ sinh viên", "mất thẻ sinh viên", "nhặt được đồ vật", "đánh rơi đồ vật", "mất đồ vật"
]

In [253]:
def normalize_text(s: str) -> str:
    s = str(s) if s is not None else ""
    s = s.lower().strip()
    # normalize common teen abbreviations to reduce false academic matches
    s = s.replace(" e ", " em ").replace(" mk ", " mình ").replace(" mn ", " mọi người ")
    return s


def contains_oos_keyword(s: str) -> bool:
    s = normalize_text(s)
    return any(k in s for k in OOS_KEYWORDS)

In [254]:
@dataclass
class Thresholds:
    min_sim: float = 0.4          # base minimum similarity required
    margin: float = 0.04           # best - second_best must exceed this, unless high_conf triggered
    high_conf: float = 0.55        # if best >= high_confident, accept even if margin small
    oos_guard_delta: float = 0.02  # if oos_similarity >= best_similarity - delta => Unknown

In [255]:
def build_topic_embeddings(
    model: SentenceTransformer,
    taxonomy: Dict[str, Dict[str, List[str]]],
    topic_ids: List[str],
    batch_size: int = 64,
) -> Dict[str, np.ndarray]:
    texts: List[str] = []
    keys: List[str] = []
    for tid in topic_ids:
        anchors = taxonomy[tid].get("anchors", [])
        if not anchors:
            raise ValueError(f"Topic {tid} must have non-empty anchors.")
        for a in anchors:
            texts.append(a)
            keys.append(tid)

    embs = model.encode(texts, batch_size=batch_size, normalize_embeddings=True, show_progress_bar=True)
    topic_vecs: Dict[str, List[np.ndarray]] = {tid: [] for tid in topic_ids}
    for tid, v in zip(keys, embs):
        topic_vecs[tid].append(v)

    out: Dict[str, np.ndarray] = {}
    for tid in topic_ids:
        out[tid] = np.mean(np.stack(topic_vecs[tid], axis=0), axis=0)
    return out

In [256]:
def build_oos_embedding(model: SentenceTransformer, anchors: List[str]) -> np.ndarray:
    embs = model.encode(anchors, batch_size=32, normalize_embeddings=True, show_progress_bar=False)
    return np.mean(np.stack(embs, axis=0), axis=0)

In [257]:
def score_topics(
    model: SentenceTransformer,
    text: str,
    topic_embeddings: Dict[str, np.ndarray],
    topic_ids: List[str],
) -> List[Tuple[str, float]]:
    x = model.encode([text], normalize_embeddings=True, show_progress_bar=False)
    topic_matrix = np.stack([topic_embeddings[tid] for tid in topic_ids], axis=0)
    sims = cosine_similarity(x, topic_matrix)[0]  # keep sklearn cosine
    scored = [(tid, float(s)) for tid, s in zip(topic_ids, sims)]
    scored.sort(key=lambda t: t[1], reverse=True)
    return scored

In [258]:
def classify(
    model: SentenceTransformer,
    text: str,
    topic_embeddings: Dict[str, np.ndarray],
    oos_embedding: np.ndarray,
    thresholds: Thresholds,
) -> Tuple[str, List[Tuple[str, float]], float]:
    norm = normalize_text(text)

    # Hard OOS veto first (prevents many false academic hits)
    if contains_oos_keyword(norm):
        scored = score_topics(model, norm, topic_embeddings, TOPIC_ORDER)
        oos_sim = float(cosine_similarity(model.encode([norm], normalize_embeddings=True, show_progress_bar=False),
                                          np.stack([oos_embedding], axis=0))[0][0])
        return "T0_OUT_OF_SCOPE", scored, oos_sim

    scored = score_topics(model, norm, topic_embeddings, TOPIC_ORDER)
    best_tid, best_sim = scored[0]
    second_sim = scored[1][1] if len(scored) > 1 else -1.0
    margin = best_sim - second_sim

    # OOS similarity guard (semantic out-of-scope)
    x = model.encode([norm], normalize_embeddings=True, show_progress_bar=False)
    oos_sim = float(cosine_similarity(x, np.stack([oos_embedding], axis=0))[0][0])

    # Accept if confident enough AND not semantically close to OOS
    confident = (best_sim >= thresholds.high_conf) or (best_sim >= thresholds.min_sim and margin >= thresholds.margin)
    if (not confident) or (oos_sim >= best_sim - thresholds.oos_guard_delta):
        return "T0_OUT_OF_SCOPE", scored, oos_sim

    return best_tid, scored, oos_sim


In [259]:
def load_model(model_id: str, cache_dir: Optional[str], local_files_only: bool) -> SentenceTransformer:
    kwargs = {}
    if cache_dir:
        kwargs["cache_folder"] = cache_dir
    if local_files_only:
        kwargs["local_files_only"] = True

    return SentenceTransformer(model_id, **kwargs)

In [260]:
def main():
    args = {
        "input": INPUT_FILE,
        "output": OUTPUT_FILE,
        "model": MODEL,
        "cache_dir": CACHE_DIR,     # ← THÊM DÒNG NÀY
        "local_only": LOCAL_ONLY,   # ← THÊM DÒNG NÀY
        "min_sim": MIN_SIM,
        "margin": MARGIN,
        "high_conf": HIGH_CONF,
        "oos_guard_delta": OOS_GUARD_DELTA,
        "batch_size": BATCH_SIZE,
    }

    df = pd.read_excel(args["input"])

    # Rename for standardized interface
    if "index" in df.columns and "id" not in df.columns:
        df = df.rename(columns={"index": "id"})
    if "segment_text" in df.columns and "text" not in df.columns:
        df = df.rename(columns={"segment_text": "text"})

    if "text" not in df.columns:
        raise ValueError("Input must contain a 'text' column (or 'segment_text' to be renamed).")

    thresholds = Thresholds(
        min_sim=args["min_sim"],
        margin=args["margin"],
        high_conf=args["high_conf"],
        oos_guard_delta=args["oos_guard_delta"],
    )

    model = load_model(args["model"], args["cache_dir"], args["local_only"])

    topic_embeddings = build_topic_embeddings(model, TAXONOMY_8, TOPIC_ORDER, batch_size=args["batch_size"])
    oos_embedding = build_oos_embedding(model, OOS_ANCHORS)

    labels: List[str] = []
    best_scores: List[float] = []
    oos_scores: List[float] = []
    all_scores_list: List[List] = []

    for t in df["text"].fillna("").astype(str).tolist():
        label, scored, oos_sim = classify(model, t, topic_embeddings, oos_embedding, thresholds)

        labels.append(label)
        best_scores.append(scored[0][1] if scored else -1.0)
        oos_scores.append(oos_sim)

        # Keep all topic scores (8 topics) sorted desc
        all_scores_list.append(
            [{"topic": tid, "score": round(score, 6)} for tid, score in scored]
        )

    df["topic_label"] = labels
    df["best_topic_score"] = best_scores
    df["oos_score"] = oos_scores
    df["all_topics_score"] = all_scores_list

    # Save
    os.makedirs(os.path.dirname(os.path.abspath(args["output"])), exist_ok=True)
    records = df.to_dict(orient="records")

    with open(args["output"], "w", encoding="utf-8") as f:
        json.dump(records, f, ensure_ascii=False, indent=2)

    print(f"Saved JSON to: {args['output']}")

if __name__ == "__main__":
    main()


No sentence-transformers model found with name VoVanPhuc/sup-SimCSE-VietNamese-phobert-base. Creating a new one with mean pooling.
c:\Users\nguye\.conda\envs\myenv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Batches: 100%|██████████| 1/1 [00:00<00:00, 37.23it/s]


Saved JSON to: output/output_supertest_confessions_of_hnmu.json


In [261]:
import json
import pandas as pd
import os

# ===== Config =====
INPUT_JSON = "output/output_supertest_confessions_of_hnmu.json"
OUTPUT_XLSX = "output/output_supertest_confessions_of_hnmu.xlsx"

# ===== Load JSON =====
with open(INPUT_JSON, "r", encoding="utf-8") as f:
    data = json.load(f)

# ===== Convert to DataFrame =====
df = pd.DataFrame(data)

# ===== Chỉ giữ 2 cột cần thiết =====
required_cols = ["text", "topic_label"]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}")

df_out = df[required_cols]

# ===== Save to Excel =====
os.makedirs(os.path.dirname(os.path.abspath(OUTPUT_XLSX)), exist_ok=True)
df_out.to_excel(OUTPUT_XLSX, index=False)

print("Saved Excel to:", OUTPUT_XLSX)


Saved Excel to: output/output_supertest_confessions_of_hnmu.xlsx
